In [1]:
import spacy
import pandas as pd
from scispacy_fun.CustomSpacyF import  CustomSpacyF 

## load sicspacy's model

In [2]:
strVer = "10"
csf  = CustomSpacyF()

##  get single word's vector from scispacy model

In [ ]:
dicVector = csf.getVector("psychosis")

for word,vec in dicVector.items():
    print(word,vec)

##  get sentence's vector from scispacy model


In [ ]:
print(csf.getSentenceVector("heart type disease"))
print(csf.getSentenceVector("neurological type disease"))

## connect to mimic db

In [ ]:
def getConnString(path:str = './db_conn.txt')->dict:
    dic_conn_str = {}

    f = open('./db_conn.txt', 'r')

    lst_tmp = []
    for line in f.readlines():
        lst_tmp = line.split(':')
        dic_conn_str[lst_tmp[0]] = lst_tmp[1].replace('\n','')

    f.close()
    del lst_tmp

    print(dic_conn_str)
    return dic_conn_str

In [ ]:
import psycopg2

dic = getConnString(path='db_conn.txt')

conn_string = f"host={dic['host']} dbname={dic['dbname']} user={dic['user']} password={dic['password']}"

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

def excuteSqlToDict(cursor,strSql):
    dic = {}

    strKey = ""
    strVal = ""

    cursor.execute(strSql)

    for row in cursor:
        strKey = row[0].strip()
        strVal = row[1].strip()
        dic[strKey] = strVal
    return dic


## get disease feature(long title) by icd code from mimic db

In [ ]:
def excuteSqlToDict(cursor,strSql):
    dic = {}

    strKey = ""
    strVal = ""

    cursor.execute(strSql)

    for row in cursor:
        strKey = row[0].strip()
        strVal = row[1].strip()
        dic[strKey] = strVal
    return dic


In [ ]:
def getIcdCodeByVersion(cursor,icd_code_list):
    strSql = """
        select
            icd_code,long_title 
        from 
            mimic_hosp.d_icd_diagnoses
        where
            icd_version = """+ strVer +"""
        and
            icd_code in (""" + icd_code_list + """)
        order by icd_code asc
    """

    return excuteSqlToDict(cursor,strSql)



## get input data by funtion: "getIcdCodeByVersion"

In [ ]:
lstIcdCode = []
f = open('icdCodeBySubId_10.txt')

strIcdCodeList = ""
for s in f.readlines():
    lst = s.split(":")
    if len(lst)== 2:
        lst = lst[1].split(",")
        for icdCode in lst:
            
            strTmp = icdCode.strip() 
            if strTmp not in strIcdCodeList:
                strIcdCodeList = strIcdCodeList + "'" + strTmp + "',"
                lstIcdCode.append(strTmp)
            # else:
            #     print(icdCode)

strIcdCodeList = strIcdCodeList[0:len(strIcdCodeList)-1]

print(len(strIcdCodeList.split(",")))
print(len(lstIcdCode))

In [ ]:
dicIcdInfo = getIcdCodeByVersion(cursor,strIcdCodeList)

In [ ]:
print(len(dicIcdInfo))
for key,val in dicIcdInfo.items():
    print(key,val)
    break


## get disease feature's vector and export to csv

In [ ]:
dicVector = {}
dicTmp = {}
aryVec = []

from datetime import datetime
strDate = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
strFileName = "icd"+strVer+"_vec_" + strDate + ".txt"

f = open(strFileName,"w")

intCount = 0
for icdCode,longTitle in dicIcdInfo.items():

    aryVec = csf.getSentenceVector(longTitle,blnStayComma = False,blnStayBrackets = False)

    strNewTitle = aryVec[0]
    odjVec = aryVec[1]

    #s = icdCode + "," + strNewTitle + "," + str(odjVec)
    s = icdCode + "," + str(odjVec)
    print(intCount,s)
    if intCount < 10:
        f.write(s + "\n")
    intCount += 1

print('count:',intCount)

f.close()

    